In [24]:
import os
import json
import getpass
from google.colab import userdata
from groq import Groq
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
#Mock tool
def get_bitcoin_price():
    """Mock function to simulate fetching live Bitcoin price data."""
    return {"price": "$98,432.10", "currency": "USD", "status": "success"}

available_functions = {
    "get_bitcoin_price": get_bitcoin_price,
}

In [25]:

MODEL_CONFIG = {
    "technical": {
        "system_prompt": "You are a Senior Python Dev. Provide detailed code solutions with docstrings and recursive logic where applicable.",
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.0
    },
    "billing": {
        "system_prompt": "You are a Billing Specialist. Be empathetic and focus on financial policy and subscription management.",
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.2
    },
    "tool": {
        "system_prompt": "You are a Real-Time Data Assistant. You MUST use the provided tools to get current prices before answering.",
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.0,
        "tools": [
            {
                "type": "function",
                "function": {
                    "name": "get_bitcoin_price",
                    "description": "Get the current market price of Bitcoin (BTC).",
                    "parameters": {
                        "type": "object",
                        "properties": {},
                    },
                },
            }
        ]
    },
    "general": {
        "system_prompt": "You are a helpful and witty general assistant named Gemini-MoE.",
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.7
    }
}

In [30]:
# 4. The Router (Intent Classification)
def route_prompt(user_input):
    """Classifies the query into: technical, billing, tool, or general."""
    routing_instructions = f"""
    Return ONLY the word 'technical', 'billing', 'tool', or 'general'.
    - technical: recursion, python, code, bugs.
    - billing: refund, charge, subscription, payment.
    - tool: price of bitcoin, btc value.
    - general: greetings, jokes, or other info.

    Input: {user_input}
    """
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": routing_instructions}],
        model="llama-3.3-70b-versatile",
        temperature=0.0,
        max_tokens=10
    )
    return response.choices[0].message.content.strip().lower()



In [31]:
# 5. The Orchestrator (Expert Execution)
def process_request(user_input):
    category = route_prompt(user_input)
    config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])

    print(f"--- DEBUG: Routing to [{category.upper()}] Expert ---")

    # Prepare arguments for the API call
    kwargs = {
        "model": config["model"],
        "messages": [
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        "temperature": config["temperature"]
    }

    # Only add tools and tool_choice if the expert has tools defined
    if "tools" in config:
        kwargs["tools"] = config["tools"]
        kwargs["tool_choice"] = "auto"

    response = client.chat.completions.create(**kwargs)
    response_message = response.choices[0].message

    # Handle Function Calling
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            print(f"--- DEBUG: Executing Tool [{function_name}] ---")

            tool_output = available_functions[function_name]()

            # Final completion with tool data
            final_response = client.chat.completions.create(
                model=config["model"],
                messages=[
                    {"role": "system", "content": config["system_prompt"]},
                    {"role": "user", "content": user_input},
                    response_message,
                    {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": function_name,
                        "content": json.dumps(tool_output),
                    },
                ],
            )
            return final_response.choices[0].message.content

    return response_message.content

In [32]:

queries = [
    "Write a recursive Python function to reverse a string.",
    "I've been charged twice for my premium plan!",
    "What is the current price of Bitcoin?",
    "Tell me a joke about robots."
]

for q in queries:
    print(f"\nUser: {q}")
    print(f"Expert: {process_request(q)}")
    print("-" * 30)


User: Write a recursive Python function to reverse a string.
--- DEBUG: Routing to [TECHNICAL] Expert ---
Expert: ### Recursive String Reversal Function
#### Overview
This function uses recursion to reverse a given string. It works by taking the first character of the string and appending it to the reversed version of the rest of the string.

#### Code
```python
def reverse_string(s: str) -> str:
    """
    Recursively reverses a given string.

    Args:
        s (str): The string to be reversed.

    Returns:
        str: The reversed string.
    """
    # Base case: if the string is empty or only contains one character, return it as is
    if len(s) <= 1:
        return s
    # Recursive case: reverse the rest of the string and append the first character
    else:
        return reverse_string(s[1:]) + s[0]

# Example usage:
def main():
    original_string = "Hello, World!"
    reversed_string = reverse_string(original_string)
    print(f"Original string: {original_string}")
    p